# Projeto de Estatística Computacional

Docente: Paulo Renato Alves Firmino <br>
Discente: Cauã Gomes Xavier <br>
Disciplina: Estatística Computacional <br>
Curso: Matemática Computacional <br>

# 1. Introdução

Este projeto tem como objetivo analisar um conjunto de dados sobre imóveis e desenvolver um modelo de aprendizado de máquina para prever o preço de venda de uma casa com base em suas características. As informações foram obtidas a partir de uma competição do Kaggle, uma plataforma de desafios e competições em ciência de dados. (https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/data). <br>
O mercado imobiliário é influenciado por diversos fatores além do número de quartos ou da presença de cozinhas. Elementos como a qualidade do material de construção, a localização e até detalhes estruturais menos óbvios podem impactar significativamente o valor final de uma propriedade. Portanto, compreender esses fatores e como eles afetam o preço das casas é essencial para compradores, vendedores e investidores tomarem decisões mais informadas. <br>
O desafio central deste projeto é prever o preço de uma casa com base em diversas variáveis, como o tamanho do terreno, a qualidade dos materiais, a condição geral da propriedade, o ano de construção, a presença de um porão e outras características estruturais.

# 2. Importações e funções

In [ ]:
import pandas as pd
import seaborn as sn
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
import math
import networkx as nx
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator
from IPython.display import display
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline 
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor
import warnings

In [ ]:
warnings.filterwarnings("ignore")

def histograma(dataFrame, nomeVariavel, qntdCategorias):
    
    distribuicaoDeFrequencia(dataFrame, nomeVariavel, qntdCategorias)
    fig, axs = plt.subplots(1, 2, figsize=(12, 4))
    dataFrame[nomeVariavel].plot(kind='box', ax=axs[0], title='')
    sn.histplot(dataFrame[nomeVariavel], color='green', bins=qntdCategorias, ax=axs[1])
    axs[1].set_ylabel(nomeVariavel)
    plt.tight_layout()
    plt.show()

def distribuicaoDeFrequencia(dataFrame, nomeVariavel, qntdCategorias):
    
    coluna = dataFrame[nomeVariavel]
    frequencia = coluna.value_counts(bins=qntdCategorias, 
                            sort=False)
    display(pd.DataFrame(frequencia))

def graficoQualitativo(dataframe, var, ax=None):
    
     dados_qualitativa = dataframe[var].value_counts(normalize = True).reset_index(name='Frequência').rename(columns={'index':var})
     display(dados_qualitativa)
     
     if ax is None:
          fig, ax = plt.subplots(figsize=(12, 4))
     dados_grafico = dataframe[var].value_counts()
     dados_grafico.plot(kind='bar', ax=ax)
     
     plt.xlabel('Valores')
     plt.ylabel('Counts')
     plt.title(f'Gráfico da Frequência: {var}')

def graficoDiscreto(dataframe, coluna, titulo="Distribuição Discreta", xlabel="frequências", ylabel="Frequência relativa"):
   
    valores, frequencias = np.unique(dataframe[coluna].dropna(), return_counts=True) 
    probabilidades = frequencias / sum(frequencias)  
    
    plt.figure(figsize=(8, 6))
    plt.vlines(valores, 0, probabilidades, colors='blue', lw=2)  
    plt.scatter(valores, probabilidades, color='red', s=30)  
    
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(f"{titulo}: {coluna}")
    plt.xticks(valores)  
    plt.grid(axis="y", linestyle="--", alpha=0.7)  

    plt.show()

def graficoQualitativoOrdinal(dataframe, coluna, xlabel="frequências", ylabel="Frequência relativa"):
    
    mapeamento = {
        "Ex": "Excellent",
        "Gd": "Good",
        "TA": "Average",
        "Fa": "Fair",
        "Po": "Poor"
    }
    
    ordem_categorias = ["Excellent", "Good", "Average", "Fair", "Poor"]
    
    dataframe[coluna] = dataframe[coluna].replace(mapeamento)
    
    dataframe[coluna] = pd.Categorical(dataframe[coluna], categories=ordem_categorias, ordered=True)
    
    dfDegrad = dataframe[coluna].value_counts(normalize=True).sort_index()
    
    qualiPlot = dfDegrad.plot(kind='bar', figsize=(12, 4), title='', fontsize=10)
    qualiPlot.set_xlabel(xlabel)
    qualiPlot.set_ylabel(ylabel)

    plt.show()

def graficoBivariado(dataframe, colunaQualitativa, colunaQuantitativa, numBins=5):

    valorMin = dataframe[colunaQuantitativa].min()
    valorMax = dataframe[colunaQuantitativa].max()
    binEdges = np.linspace(valorMin, valorMax, numBins + 1).astype(int) 

    dataframe['Prices'] = pd.cut(dataframe[colunaQuantitativa], bins=binEdges, include_lowest=True, right=False)

    tabelaContingencia = pd.crosstab(index=dataframe[colunaQualitativa], 
                                      columns=dataframe['Prices'], 
                                      normalize="index")

    tituloTabela = f"Distribuição de {colunaQuantitativa} (Agrupado) por {colunaQualitativa}"
    print("\n" + "-" * 50)
    print(f"{tituloTabela}")
    print("-" * 50)
    display(tabelaContingencia)

    plt.figure(figsize=(8,5))
    tabelaContingencia.plot(kind="bar", stacked=False, colormap="tab10", figsize=(8,5))
    
    tituloGrafico = f"{colunaQuantitativa} (Agrupado) por {colunaQualitativa}"
    plt.title(tituloGrafico, fontsize=14)
    plt.xlabel(colunaQualitativa, fontsize=12)
    plt.ylabel("Proporção", fontsize=12)
    plt.xticks(rotation=0)
    plt.legend(title=f"{colunaQuantitativa} (Agrupado)")
    plt.grid(axis="y", linestyle="--", alpha=0.7)
    
    plt.show()

    dataframe.drop(columns=['Prices'], inplace=True)


def medidasDispersaoPosicao(dataframe, coluna):

    media = dataframe[coluna].mean()
    moda = dataframe[coluna].mode().values
    mediana = dataframe[coluna].median()
    desvio_padrao = dataframe[coluna].std()
    variancia = dataframe[coluna].var()

    estatisticas_df = pd.DataFrame({
        "Métrica": ["Média", "Moda", "Mediana", "Desvio Padrão", "Variância"],
        "Valor": [media, moda, mediana, desvio_padrao, variancia]
    })

    from IPython.display import display
    display(estatisticas_df)

def transformarEmQualitativo(coluna):
    
    mapeamento = {
        10: "Very Excellent",
        9: "Excellent",
        8: "Very Good",
        7: "Good",
        6: "Above Average",
        5: "Average",
        4: "Below Average",
        3: "Fair",
        2: "Poor",
        1: "Very Poor"
    }
    return coluna.map(mapeamento).fillna("Desconhecido") 

def transformarAnoEmCategoria(dataframe, coluna):
   
    def categorizarAno(ano):
        if ano < 1900:
            return "Muito antiga"
        elif 1900 <= ano < 1950:
            return "Antiga"
        elif 1950 <= ano < 2000:
            return "Média idade"
        else:
            return "Nova"
    
    dataframe[coluna + 'Categoria'] = dataframe[coluna].apply(categorizarAno)

def transformarPrecoEmCategoria(dataframe, coluna):
   
    def categorizarPreco(preco):
        if preco < 214925:
            return "Baixo"
        elif 214925 <= preco < 394950:
            return "Médio baixo"
        elif 394950 <= preco < 574975:
            return "Médio alto"
        else:
            return "Alto"
    
    dataframe[coluna + 'Categoria'] = dataframe[coluna].apply(categorizarPreco)
    
def avaliarModelo(model_name, y_true, y_pred):
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
    import numpy as np

    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
   
    print(f"{model_name:<15} RMSE =     {rmse:.15f}")
    print(f"{model_name:<15} MAE =      {mae:.15f}")
    print(f"{model_name:<15} MAPE =     {mape:.2f}%")
    print(f"{model_name:<15} R^2 =      {r2:.15f}")

# 3. Pré-processamento

O pré-processamento transforma dados brutos em um formato padronizado e relevante, facilitando a realização das análises, além de melhorar a qualidade dos resultados obtidos.

## 3.1. Análise descritiva dos dados

- Ao analisar as colunas abaixo, observa-se que algumas colunas apresentam um grande número de valores indefinidos (N/A). Por esse motivo, é importante removê-las da tabela, a fim de evitar interferências na extração de informações úteis dos dados. Essa abordagem contribui para uma análise mais precisa e confiável.

In [ ]:
dadosTreino = pd.read_csv(filepath_or_buffer = "database/train.csv", 
                    sep=",")

dadosTreino.info()

- Conferindo as colunas que têm mais da metade dos valores como indefinidos (N/A) e tirando elas da tabela.

In [ ]:
colunasNa = ['Alley', 'PoolQC', 'Fence', 'MiscFeature', 'MasVnrType']

totalLinhasDataFrame = len(dadosTreino)
print(f"Total de linhas no DataFrame: {totalLinhasDataFrame}")

for colunaNa in colunasNa:
    quantidadeNa = dadosTreino[colunaNa].isna().sum()
    print(f"Total de valores N/A na coluna '{colunaNa}': {quantidadeNa}")
    dadosTreino.drop(columns=[colunaNa],inplace=True)

- A tabela abaixo representa o conjunto de dados. Observa-se os dados abrangem diversas informações sobre cada imóvel, incluindo planicidade do local onde a propriedade foi construida e área da varanda(em pés quadrados). Com base nessas informações, é possível obter uma visão mais ampla e detalhada dos dados, o que será explorado nas próximas etapas.

In [ ]:
dadosTreino.drop(columns=["Id"],inplace=True)

dadosTreino.head(5)


As principais **variáveis quantitativas** de interesse e como estas se relacionam com nossa variável depedente ("SalePrice"):<br>
- **YearBuilt**: Ano original da contrução. 
- **YearRemodAdd**: Ano da última remodelação. 
- **LotArea**:  Área do lote(em pés quadrados).
- **GarageArea**: Área da garagem(em pés quadrados). 
- **PoolArea**: Área da piscina(em pés quadrados).
- **Fireplaces**: Quantidade de lareiras.
- **OverallQual**: Qualidade geral do material e do acabamento(varia de 1 a 10).
- **TotalBsmtSF**: Área do porão(em pés quadrados).
- **FullBath**: Quantidade de banheiros completos.
      

é possível entender melhor a relação entre as variáveis citadas acima e a variável dependente. Para isso, utiliza-se o __Coeficiente de Correlação de Pearson__. Para isso, é necessário criar uma cópia dos dados retirando as variáveis qualitativas. <br>
A correlação de Pearson é um método estatístico que avalia a relação entre duas variáveis quantitativas. Seu valor está compreendido no intervalo de -1 a +1. <br>
Um valor igual a 0 indica ausência de associação linear entre as variáveis. Um valor maior que 0 revela que, à medida que uma variável aumenta, a outra também tende a aumentar, caracterizando uma correlação positiva. Por outro lado, um valor menor que 0 demonstra uma correlação negativa entre as variáveis, indicando que, à medida que uma variável cresce, a outra tende a diminuir. 
Apartir disso é possivel fazer a seguinte análise:

- A qualidade geral do material e do acabamento é o fator mais importante para definir o preço do imóvel.
- A área da garagem é um fator mais relevante do que o ano em que imóvel foi construido.


In [ ]:
pd.set_option('display.max_rows',20)

dadosTemporário = dadosTreino.select_dtypes(include=['int64', 'float64'])

correlacaoMatriz = dadosTemporário.corr()
correlacaoMatriz["SalePrice"].sort_values(ascending=False).head(10)

As principais **variáveis quantitativas** de interesse e como estas se relacionam com nossa variável depedente ("SalePrice"): <br>

- **Street**: Se a rua possui asfalto
- **HouseStyle**: Estilo da casa e a quantidade de andares
- **RoofStyle**: Tipo de telhado
- **ExterQual**: Qualidade do material no exterior do imóvel
- **ExterCond**: Condição atual do material no exterior do imóvel
- **Foudation**: Material usado na construção do imóvel
- **BsmtCond**: Condição atual do porão
- **Electrical**: Sistema elétrico



### Variáveis Quantitativas

- Para criar a distribuição de frequências de uma coluna com variáveis quantitativas contínuas, é necessário definir alguns parâmetros: 

In [ ]:
numeroElementos = dadosTreino['LotFrontage'].notna().sum()

print(f"Quantidade de observações: {numeroElementos}")

LotFrontageMaximo = dadosTreino['LotFrontage'].max()
print(f"Valor máximo da coluna 'LotFrontage': {LotFrontageMaximo}")

LotFrontageMinimo = dadosTreino['LotFrontage'].min()
print(f"Valor mínimo da coluna 'LotFrontage': {LotFrontageMinimo}")
  
raizQuadrada = math.sqrt(numeroElementos)  
numeroCategorias = round(raizQuadrada)

print(f"Quantidade de categorias: {numeroCategorias}")

tamanhoIntervalo = (LotFrontageMaximo - LotFrontageMinimo) / numeroCategorias

print(f"Tamanho do intervalo: {tamanhoIntervalo}")

- No gráfico univariado e na distribuição de metros lineares da rua conectada a casa há uma grande concentração entre $54 \, ft$ (ou $5 \, m$)   e $62 \, ft$ (ou $5.6 \, m$), porém a maior concentração encontra-se entre $63 \, ft$ (ou $5.8 \, m$) e $71 \, ft$ (ou $6.6 \, m$).

- O gráfico em boxplot indica que as medições abaixo de $25 \, ft$ (ou $2.3 \, m$) e acima de $122 \, ft$ (ou $11.3 \, m$) já são considerados valores atípicos.

- Sendo assim, pela análise da distribuição dos gráficos e da distribuição de frequências, a maior parte dos imóveis tem entre $50 \, ft$ (ou $4.6 \, m$) e $100 \, ft$ (ou $9.3 \, m$)

In [ ]:
pd.set_option('display.max_rows', 35) 

histograma(dadosTreino, "LotFrontage", 35)

- O gráfico abaixo revela que é incomum um imóvel ter mais de duas lareiras, sugerindo que a quantidade de lareiras pode não ser um fator determinante na definição do preço da propriedade.

In [ ]:
graficoDiscreto(dadosTreino, "Fireplaces", "Quantidade de lareiras")

### Medidas de dispersão e posição

As medidas de posição permitem estimar o valor que a variável aleatória poderá assumir, enquanto as medidas de dispersão avaliam a confiabilidade dessas estimativas. As medidas analisadas neste contexto são a moda, a média, a mediana, a variância e o desvio padrão.

A moda, a média e a mediana são medidas de posição e indicam:

- Média: obtida pela divisão da soma de todos os valores pelo número total de observações, representando o ponto de equilíbrio da amostra.
- Moda: o valor mais frequente na amostra.
- Mediana: o valor central de um conjunto de dados organizados em ordem crescente.

Já o desvio padrão e a variância são medidas de dispersão, indicando o grau de variação dos valores em relação à média, ou seja, se os dados estão mais concentrados ou dispersos.

- A mediana abaixo indica que, pelo menos a metade dos imóveis, possui uma ou nenhuma lareira. Logo, a quantidade de lareiras não deve ser muito significante para o preço da casa.

In [ ]:
medidasDispersaoPosicao(dadosTreino, "Fireplaces")

- Como há uma grande variância na área dos lotes, os valores estão distantes da média, indicando uma ampla diversidade nos dados.

In [ ]:
medidasDispersaoPosicao(dadosTreino, "LotArea")

- Como o desvio padrão da área das garagens está próximo da metade da média, isso indica que, apesar das variações, ainda há um certo padrão nos dados.

In [ ]:
medidasDispersaoPosicao(dadosTreino, "GarageArea")

- A moda da qualidade geral do material e do acabamento é 5 (mediano). Portanto, como esperado, a maioria dos imóveis não está nos extremos (excelente ou pobre).

In [ ]:
medidasDispersaoPosicao(dadosTreino, "OverallQual")

- Existem poucos imóveis em que a quantidade de banheiros é 1, o que indica uma maior preferência por casas com mais banheiros.

In [ ]:
medidasDispersaoPosicao(dadosTreino, "FullBath")

### Variáveis Qualitativas

- Com base no gráfico dos tipos de telhado dos imóveis apresentado abaixo, é possível observar que a maioria das propriedades possui telhados do tipo gable, representando 78% do total.

In [ ]:
graficoQualitativo(dadosTreino, "RoofStyle")

- A maioria dos imóveis (aproximadamente 80%) possui a qualidade do material externo classificada como "mediana". Além disso, há uma quantidade insignificante de imóveis com qualidade "falha" ou "Pobre".

In [ ]:
graficoQualitativoOrdinal(dadosTreino, "ExterQual", "Qualidade do material externo")

- Ao analisar a distribuição do preço de venda (SalePrice) por estilo de telhado (RoofStyle), é possivel observa que certos estilos apresentam uma maior concentração em faixas de preço específicas.
- O estilo Hip é o mais diversificado, abrangendo todas as faixas de preço, embora sua maior concentração esteja na faixa mais baixa (34.900 - 178.920), com mais ou menos 58%. Já o estilo Flat tem sua distribuição mais equilibrada entre as faixas inferiores, mas não aparece nas faixas mais altas.
- O estilo Shed é o mais restrito, aparecendo apenas na segunda faixa de preço (178.920 - 322.940), com 100% de sua presença ali. Isso sugere que casas com esse tipo de telhado podem ter um padrão de preço bem específico.
- Então, há uma correlação entre o tipo de telhado e a faixa de preço da casa, indicando que certos estilos são mais predominantes em determinados níveis de valor.

In [ ]:
graficoBivariado(dadosTreino, "RoofStyle", "SalePrice")

## 3.2. Transformação de Variáveis

- A seguir, foram selecionadas as variáveis com base na análise descritiva realizada, considerando apenas aquelas submetidas à avaliação. Essas variáveis serão utilizadas no modelo preditivo, com o objetivo de estimar a variável "SalePrice".

In [ ]:
quantVars  = ["YearBuilt",
               "YearRemodAdd", 
               "LotArea", 
               "GarageArea",
               "PoolArea",
               "Fireplaces",
               "OverallQual",
               "TotalBsmtSF",
               "FullBath",
               "GarageYrBlt",
               "MasVnrArea",
               "LotFrontage"] 

qualiVars = ["Street",
              "HouseStyle",
              "RoofStyle",
              "ExterQual",
              "ExterCond",
              "Electrical",
              "BsmtQual", 
              "BsmtCond",
              "BsmtExposure",
              "BsmtFinType1",
              "BsmtFinType2",
              "FireplaceQu",
              "GarageType"]

quantiDf = dadosTreino[quantVars]
qualiDf = dadosTreino[qualiVars]

x = dadosTreino[quantVars + qualiVars]
y = dadosTreino['SalePrice']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

- Como as colunas possuem dados é necessário imputar. Nesse caso imputar utilizando a mediana.

In [ ]:
print("-----------------dados quantitativos antes da imputação----------------")
print("\n")
display(quantiDf)
imputer = SimpleImputer(strategy = "median", add_indicator = True)
imputer.fit(quantiDf)

- É necessário realizar a padronização das variáveis quantitativas, isto é, colocar os valores das variáveis em uma mesma escala, para que tenham média 0 e desvio padrão de 1. Este processo é crucial, pois para muitos modelos de machine learning os dados precisam estar padronizados.

In [ ]:
scaler = StandardScaler() 
scaler.fit(quantiDf)
quantiDf = pd.DataFrame(scaler.transform(quantiDf), columns= scaler.get_feature_names_out())
print("---------------Dados apos a padronização-----------------")
display(quantiDf.head())

## 2.4. Partição dos dados em treinamento e teste

- Nesta etapa, realiza-se uma divisão do conjunto de dados com o objetivo de aplicá-lo aos modelos que serão desenvolvidos. Essa divisão é fundamental para o ajuste adequado dos modelos, para a prevenção do overfitting. Além disso, será utilizado o sistema de pipeline, que consiste em uma cadeia de etapas que sintetiza todo o processo de preparação e padronização dos dados executado previamente.

In [ ]:
trainOk = pd.concat([quantiDf, qualiDf], axis=1)
trainOk.head()

In [ ]:
quantiPipeline = Pipeline([('imputer', SimpleImputer(strategy = "median", add_indicator = True)), 
                           ('std_scaler', StandardScaler())]) 

qualiPipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="most_frequent")),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))    
])

- Dataframe após todas as transformações.

quantiOk = pd.DataFrame(quantiPipeline.fit_transform(quantiDf), 
                        columns=quantiPipeline.get_feature_names_out())
quantiOk.head()

In [ ]:
preprocessor  = ColumnTransformer([("qt", quantiPipeline, quantVars), 
                                  ("ql", qualiPipeline, qualiVars)]) 
preprocessor 

## 4. Modelos de ML

Para analisar o desempenho dos modelos criados, existem métricas de erros: R2, MAE, RMSE, RMSLE.

- **RMSLE** (Root Mean Squared Logarithmic Error): mede a raiz quadrada da média das diferenças quadradas entre o logaritmo dos valores previstos e o logaritmo dos valores reais. Geralmente é usado quando a variável de destino possui uma ampla faixa de valores.
- **RMSE** (Root Mean Squared Error): mede a raiz quadrada da média das diferenças quadradas entre os valores previstos e os valores reais. É mais sensível a outliers do que o MAE.
- **R2** (R-quadrado) : mede o quão bem a linha de regressão ajusta os pontos de dados. Varia de 0 a 1, com 1 indicando um ajuste perfeito e 0 indicando nenhum ajuste.
- **MAE** (Mean Absolute Error): mede a diferença média absoluta entre os valores previstos e os valores reais. É menos sensível a outliers do que o RMSE.

### XGBoost Regression

In [ ]:
xgb_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(
        n_estimators=150,
        max_depth=6,
        learning_rate=0.1,
        random_state=42,
        objective='reg:squarederror'
    ))
])

xgb_pipeline.fit(x_train, y_train)

y_pred_xg = xgb_pipeline.predict(X_test)

avaliarModelo("XGBoost", y_test, y_pred_xg)

### Random Forest Regressor

In [ ]:
rf_model = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=250, random_state=42))
])

rf_model.fit(x_train, y_train)

y_pred_rf = rf_model.predict(X_test)

avaliarModelo("Random Forest", y_test, y_pred_rf)

### Support Vector Regression - SVR

In [ ]:
svr_model = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', SVR(kernel='rbf', C=100, gamma='auto'))
])

svr_model.fit(x_train, y_train)

y_pred_svr = svr_model.predict(x_test)

avaliarModelo("SVR", y_test, y_pred_svr)

### SVR GridSearchCV

In [ ]:
pipeline_svr = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', SVR())
])

param_grid_svr = {
    'regressor__kernel': ['rbf', 'linear'],
    'regressor__C': [1, 10, 100],
    'regressor__epsilon': [0.01, 0.1, 1],
    'regressor__gamma': ['scale', 'auto']
}

grid_search = GridSearchCV(pipeline_svr, param_grid_svr, cv=5, scoring='r2', n_jobs=-1, verbose=2)

grid_search.fit(x_train, y_train.values.ravel())

print("Melhores parâmetros:", grid_search.best_params_)
print("Melhor R² no treino (cross-validation):", grid_search.best_score_)

y_pred_svr_grid = grid_search.predict(x_test)

avaliarModelo("SVR com GridSearch", y_test, y_pred_svr_grid)

### Rede Neural Artificial - ANN

In [ ]:
ann_modelo = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', MLPRegressor(max_iter=1000, random_state=42))
])

param_grid = {
    'regressor__hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],
    'regressor__activation': ['relu', 'tanh'],
    'regressor__solver': ['adam', 'sgd'],
    'regressor__alpha': [0.0001, 0.001, 0.01],
    'regressor__learning_rate': ['constant', 'adaptive']
}

grid_search = GridSearchCV(ann_modelo, param_grid, cv=5, scoring='r2', n_jobs=-1, verbose=1)
grid_search.fit(x_train, y_train)

best_model = grid_search.best_estimator_

print("🔧 Melhores parâmetros encontrados:")
print(grid_search.best_params_)
print(f"🔍 Melhor R² no treino (CV): {grid_search.best_score_:.4f}")

y_pred_test_ann_grid = best_model.predict(x_test)

avaliarModelo("ANN com GridSearch", y_test, y_pred_test_ann_grid)